In [7]:
import numpy as np # linear algebra
np.set_printoptions(linewidth=1000)
import struct
from array import array
import os

class MnistDataloader(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath
    
    def read_images_labels(self, images_filepath, labels_filepath):        
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())        
        
        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())        
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img            
        
        return images, labels
            
    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (x_train, y_train),(x_test, y_test)
%matplotlib inline
import random
import matplotlib.pyplot as plt

input_path = './archive/'
training_images_filepath = os.path.join(input_path, 'train-images-idx3-ubyte/train-images-idx3-ubyte')
training_labels_filepath = os.path.join(input_path, 'train-labels-idx1-ubyte/train-labels-idx1-ubyte')
test_images_filepath = os.path.join(input_path, 't10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
test_labels_filepath = os.path.join(input_path, 't10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')

mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(x_train, y_train), (x_test, y_test) = mnist_dataloader.load_data()

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

print(x_train.shape)
x_train = x_train.reshape(x_train.shape[0], -1)
print(x_train.shape)
x_train = x_train.astype('float32')
print(x_train[0].shape)

(60000, 28, 28)
(60000, 784)
(784,)


In [8]:
x_train[0], y_train[0]

(array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   3.,  18.,  18.,  18., 126., 136., 175.,  26., 166., 255., 247., 127.,   0.,
  

In [63]:
import math
from scipy.special import softmax

class Node:
    def __init__(self, inputs):
        self.weights = np.random.rand(inputs.shape[0])
        self.bias = np.random.rand(1)

class Layer:
    def __init__(self, layer_dim, X, afunc):
        self.X = X
        self.W = np.random.rand(X.shape[0], layer_dim) * 0.002 - 0.001
        self.B = np.zeros(layer_dim)
        self.Z = np.matmul(self.X, self.W) + self.B
        self.afunc = afunc
    
    @staticmethod
    def relu(z):
        return max(0, z)
    
    @staticmethod
    def softmax(Z):
        return np.array([math.exp(Z[i])/sum([math.exp(Z[j]) for j in range(len(Z))]) for i in range(len(Z))])
    
    def activation(self):
        function_mapping = {
            'relu': np.vectorize(self.relu),
            'softmax': softmax,
        }
        return function_mapping[self.afunc]

    def forward(self):
        return self.activation()(self.Z)

class FNN:
    def __init__(self, layers) -> None:
        self.layers = layers

l_1 = Layer(128, x_train.shape[0], 'relu')
print('X', l_1.X, l_1.X.shape)
print('W', l_1.W, l_1.W.shape)
print('B', l_1.B, l_1.B.shape)
print('Z', l_1.Z, l_1.Z.shape)
output_1 = l_1.forward()
print(output_1, output_1.shape)
print('*************')
l_2 = Layer(128, output_1, 'relu')
print('X', l_2.X, l_2.X.shape)
print('W', l_2.W, l_2.W.shape)
print('B', l_2.B, l_2.B.shape)
print('Z', l_2.Z, l_2.Z.shape)
output_2 = l_2.forward()
print(output_2, output_2.shape)
l_3 = Layer(10, output_1, 'softmax')
print('X', l_3.X, l_3.X.shape)
print('W', l_3.W, l_3.W.shape)
print('B', l_3.B, l_3.B.shape)
print('Z', l_3.Z, l_3.Z.shape)
output_3 = l_3.forward()
print(output_3, output_3.shape)
np.argmax(output_3)

SyntaxError: invalid syntax (2441916003.py, line 37)